In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import OpenAIWhisperParser
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.callbacks.base import BaseCallbackHandler


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(f"{token}", end="", flush=True)


# API 키 정보 로드
load_dotenv()

# YouTube 동영상 URL
urls = [""]

# 오디오 파일을 저장할 디렉토리
save_dir = "./youtube_audios/"

# 동영상을 텍스트로 변환
loader = GenericLoader(YoutubeAudioLoader(
    urls, save_dir), OpenAIWhisperParser())
docs = loader.load()


# 문서 결합
# docs의 각 문서에서 page_content를 추출하여 combined_docs에 저장합니다.
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)


# 텍스트를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_text(text)


# 색인을 구축합니다.
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_texts(splits, embeddings)

# QA 체인 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        model_name="gpt-4-turbo-preview",
        temperature=0,
        streaming=True,
        callbacks=[StreamCallback()],
    ),
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
)



In [ ]:
# 질문을 하세요!
query = ""
answer = qa_chain.invoke({"query": query})